In [ ]:
import os
import sys
import time
import random
import numpy as np

from matplotlib import colors
import matplotlib.pyplot as plt
from collections import namedtuple
from IPython.display import clear_output

import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.losses import *
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback, CSVLogger
from tensorflow.keras import mixed_precision


from utils.plot_utils import plot_iou_trainId, plot_iou_catId
from utils.data_utils import get_labels, parse_record, get_dataset_from_tfrecord
from models.u2net_logits import U2NET
from models.erfnet_logits import ERFNet

physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
K.clear_session()

def enable_amp():
    mixed_precision.set_global_policy("mixed_float16")

print("Tensorflow version: ", tf.__version__)
print(physical_devices,"\n")
enable_amp()

In [ ]:
!nvidia-smi -L

In [ ]:
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)

In [ ]:
fine = True

if fine:
    train_tfrecord_dir = "records/trainIds_train.record"
    test_tfrecord_dir = "records/trainIds_val.record"
else:
    train_tfrecord_dir = "records/trainIds_train_extra.record"

img_height = 512 # 512
img_width = 1024 # 1024

n_classes = 20

labels = get_labels()
trainid2label = { label.trainId : label for label in labels }
catid2label = { label.categoryId : label for label in labels }

There's a bunch of data augmentation steps here. Random cropping, as well as random shifts in brightness, saturation, contrast, and hue. 

In [ ]:
np.array([0.5, 0.5625, 0.625, 0.6875, 0.75, 0.8125, 0.875, 0.9375, 1.0]) * 1024

In [ ]:
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])


@tf.function
def random_crop(image, mask):
    """
    Inputs: full resolution image and mask
    A scale between 0.5 and 1.0 is randomly chosen. 
    Then, we multiply original height and width by the scale, 
    and randomly crop to the scaled height and width.
    """
    scales = tf.convert_to_tensor(np.array(
        [0.25, 0.3125, 0.375, 0.4375, 0.5, 0.5625, 0.625, 0.6875, 0.75, 0.8125, 0.875, 0.9375, 1.0]))
    scale = scales[tf.random.uniform(shape=[], minval=0, maxval=13, dtype=tf.int32)]
    scale = tf.cast(scale, tf.float32)
    
    shape = tf.cast(tf.shape(image), tf.float32)
    h = tf.cast(shape[0] * scale, tf.int32)
    w = tf.cast(shape[1] * scale, tf.int32)
    combined_tensor = tf.concat([image, mask], axis=2)
    combined_tensor = tf.image.random_crop(combined_tensor, size=[h, w, 4])
    return combined_tensor[:,:,0:3], combined_tensor[:,:,-1]

@tf.function
def mask_to_categorical(image, mask):
    mask = tf.squeeze(mask)
    # mask = tf.one_hot(tf.cast(mask, tf.int32), n_classes)
    # mask = tf.cast(mask, tf.float32)
    mask = tf.cast(mask, tf.int32)
    return image, mask


@tf.function
def normalize(image, mask):
    image = image / 255.0
    image = image - MEAN
    image = image / STD
    return image, mask


@tf.function
def load_image_train(input_image, input_mask):
    
    image = tf.cast(input_image, tf.uint8)
    mask = tf.cast(input_mask, tf.uint8)
    
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
         
    # if tf.random.uniform(()) > 0.5:
    #     image, mask = random_crop(image, mask)
    #     mask = tf.expand_dims(mask, axis=-1)
    
    image = tf.image.resize(image, (img_height, img_width))
    mask = tf.image.resize(mask, (img_height, img_width))
    
    image, mask = normalize(tf.cast(image, tf.float32), mask)
    
    if tf.random.uniform(()) > 0.5:
        image = tf.image.random_brightness(image, 0.05)
        image = tf.image.random_saturation(image, 0.6, 1.6)
        image = tf.image.random_contrast(image, 0.7, 1.3)
        image = tf.image.random_hue(image, 0.05)
    
    image, mask = mask_to_categorical(image, mask)
    mask = tf.squeeze(mask)

    return image, mask


def load_image_test(input_image, input_mask):
    image = tf.image.resize(input_image, (img_height, img_width))
    mask = tf.image.resize(input_mask, (img_height, img_width))
    
    image, mask = normalize(tf.cast(image, tf.float32), mask)
    image, mask = mask_to_categorical(image, mask)
    mask = tf.squeeze(mask)
    return image, mask


def load_image_eval(input_image, input_mask):
    image = tf.image.resize(input_image, (img_height, img_width))
    
    image, mask = normalize(tf.cast(image, tf.float32), input_mask)
    image, mask = mask_to_categorical(image, mask)
    mask = tf.squeeze(mask)
    return image, mask

In [ ]:
if fine:
    TRAIN_LENGTH = 2975
    TEST_LENGTH = 500
else:
    TRAIN_LENGTH = 18000
    TEST_LENGTH = 1998

BATCH_SIZE = 8
BUFFER_SIZE = 300

In [ ]:
if fine:
    train_ds = get_dataset_from_tfrecord(train_tfrecord_dir)
    test_ds = get_dataset_from_tfrecord(test_tfrecord_dir)
else:
    all_ds = get_dataset_from_tfrecord(train_tfrecord_dir)
    train_ds = all_ds.skip(TEST_LENGTH)
    test_ds = all_ds.take(TEST_LENGTH) 

In [ ]:
# Preprocessing: random crop the images and masks, flip them
train = train_ds.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = test_ds.map(load_image_test)
eval = test_ds.map(load_image_eval)

In [ ]:
train_dataset = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

In [ ]:
def label_to_rgb(mask):
    h = mask.shape[0]
    w = mask.shape[1]
    mask_rgb = np.zeros((h, w, 3), dtype=np.uint8)
    for val, key in trainid2label.items():
        indices = mask == val
        mask_rgb[indices.squeeze()] = key.color 
    return mask_rgb



def display(display_list, title=True):
    plt.figure(figsize=(15, 5))
    if title:
        title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        if title:
            plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
for image, mask in test.take(27): # c: 12 f: 16, 21, 27
    sample_image, sample_mask = image, mask

# sample_mask = tf.argmax(sample_mask, axis=-1)
sample_mask = sample_mask[..., tf.newaxis]
sample_mask = label_to_rgb(sample_mask.numpy())
display([sample_image, sample_mask])

## Model



In [ ]:
def iou_coef(y_true, y_pred):
    smooth = 1
    iou_total = 0
    for i in range(1, n_classes):
        tp = tf.math.reduce_sum(y_pred[:,:,:,i] * y_true[:,:,:,i], axis=(1,2))
        fn = tf.math.reduce_sum(y_true[:,:,:,i] * (1 - y_pred[:,:,:,i]), axis=(1,2)) 
        fp = tf.math.reduce_sum(y_pred[:,:,:,i] * (1 - y_true[:,:,:,i]), axis=(1,2)) 
        iou = tf.math.reduce_mean(tf.math.divide_no_nan(tp+smooth, tp+fn+fp+smooth), axis=0)
        iou_total += iou
    iou_macro = iou_total / (n_classes - 1)
    return iou_macro


def iou_coef_sparse(y_true, y_pred):
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), n_classes)
    y_pred = tf.nn.softmax(y_pred, axis=-1)
    smooth = 1
    iou_total = 0
    for i in range(1, n_classes):
        tp = tf.math.reduce_sum(y_pred[:,:,:,i] * y_true[:,:,:,i], axis=(1,2))
        fn = tf.math.reduce_sum(y_true[:,:,:,i] * (1 - y_pred[:,:,:,i]), axis=(1,2)) 
        fp = tf.math.reduce_sum(y_pred[:,:,:,i] * (1 - y_true[:,:,:,i]), axis=(1,2)) 
        iou = tf.math.reduce_mean(tf.math.divide_no_nan(tp+smooth, tp+fn+fp+smooth), axis=0)
        iou_total += iou
    iou_macro = iou_total / (n_classes - 1)
    return iou_macro


def iou_micro_coef(y_true, y_pred):
    """Gets the global IoU score (averaged over the batch) over all the classes at once"""
    smooth = 1.0
    A = y_true[:,:,:,1:]
    B = y_pred[:,:,:,1:]
    intersection = tf.math.reduce_sum(A * B, axis=(1,2,3))
    union = tf.math.reduce_sum(A, axis=(1,2,3)) + tf.math.reduce_sum(B, axis=(1,2,3)) - intersection
    iou = tf.math.reduce_mean(tf.math.divide_no_nan(intersection + smooth, union + smooth), axis=0)
    return iou

In [ ]:
c_weights = [0.0,    2.602,  6.707,  3.522,  9.877, 
             9.685,  9.398,  10.288, 9.969,  4.336, 
             9.454,  7.617,  9.405,  10.359, 6.373, 
             10.231, 10.262, 10.264, 10.394, 10.094] 


cce_loss = tf.keras.losses.CategoricalCrossentropy()


def cce_iou_micro_loss(y_true, y_pred):
    return (cce_loss(y_true, y_pred) - iou_micro_coef(y_true, y_pred)) + 1


def weighted_cross_entropy_loss(y_true_labels, y_pred_logits):
    # y_true_labels: (batch_size, img_h, img_w)
    # y_pred_logits: (batch_size, img_h, img_w, num_classes)
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true_labels, logits=y_pred_logits)  # (batch_size, img_h, img_w)
    weights = tf.gather(c_weights, y_true_labels)  # (batch_size, img_h, img_w)
    losses = tf.multiply(losses, weights)
    return tf.reduce_mean(losses)

In [ ]:
K.clear_session()
# model = U2NET(input_height=img_height, input_width=img_width, n_classes=n_classes)
# model = HRNet(input_height=img_height, input_width=img_width, n_classes=n_classes, W=40)
model = ERFNet(input_height=img_height, input_width=img_width, n_classes=n_classes)

In [ ]:
# plot_model(model, show_shapes=True, dpi=64)
model.summary()

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.squeeze(pred_mask)
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    pred_mask = label_to_rgb(pred_mask.numpy())
    return pred_mask


def show_predictions():
    pred_mask = model.predict(sample_image[tf.newaxis, ...])
    if "U2Net" in model.name:
        pred_mask = pred_mask[0]
    display([sample_image, sample_mask, create_mask(pred_mask)])


class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
if fine:
    MODEL_PATH = "weights/"+model.name+"_512x1024.h5"
else:
    MODEL_PATH = "weights/"+model.name+"_coarse_512x1024.h5"

# model.load_weights(MODEL_PATH)
# model.load_weights("weights/U2Net.h5")

In [ ]:
show_predictions()

In [ ]:
if fine:
    EPOCHS = 100
else:
    EPOCHS = 40

In [ ]:
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE 
VALIDATION_STEPS = TEST_LENGTH // BATCH_SIZE 
TOTAL_UPDATE_STEPS = int(EPOCHS * STEPS_PER_EPOCH)

In [ ]:
learning_rate_fn = PolynomialDecay(initial_learning_rate = 8e-4, decay_steps = TOTAL_UPDATE_STEPS, 
                                   end_learning_rate = 1e-8, power = 0.9)

model.compile(
    optimizer = Adam(learning_rate=learning_rate_fn, decay=0.0005), 
    loss = weighted_cross_entropy_loss, 
    metrics = ['accuracy', iou_coef_sparse]
)

In [ ]:
callbacks = [
    DisplayCallback(),
    ModelCheckpoint(MODEL_PATH, monitor='val_iou_coef_sparse', verbose=0, save_best_only=True, save_weights_only=True, mode='max'),
]

In [ ]:
results = model.fit(
    train_dataset,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    epochs = (EPOCHS*SUBSPLITS),
    validation_data = test_dataset,
    callbacks = callbacks, 
    verbose = 1  
)

In [ ]:
results = model.history

In [ ]:
def plot_history(results, model):
         
    plt.figure(figsize=(16,7))
    plt.subplot(1,3,1)
    if "U2Net" in model.name:
        plt.plot(results.history['d0_loss'], 'r', label='Training loss')
        plt.plot(results.history['val_d0_loss'], 'b', label='Validation loss')
    else:
        plt.plot(results.history['loss'], 'r', label='Training loss')
        plt.plot(results.history['val_loss'], 'b', label='Validation loss')
    plt.title("Loss: "+model.name, fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.legend(prop={'size': 14})

    plt.subplot(1,3,2)
    if "U2Net" in model.name:
        plt.plot(results.history['d0_accuracy'], 'r', label='Training accuracy')
        plt.plot(results.history['val_d0_accuracy'], 'b', label='Validation accuracy')
    else:
        plt.plot(results.history['accuracy'], 'r', label='Training accuracy')
        plt.plot(results.history['val_accuracy'], 'b', label='Validation accuracy')
    plt.title('Accuracy: '+model.name, fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.legend(prop={'size': 14})

    plt.subplot(1,3,3)
    if "U2Net" in model.name:
        plt.plot(results.history['d0_iou_coef'], 'r', label='IoU coefficient')
        plt.plot(results.history['val_d0_iou_coef'], 'b', label='Validation IoU coefficient')
    else:
        plt.plot(results.history['iou_coef'], 'r', label='IoU coefficient')
        plt.plot(results.history['val_iou_coef'], 'b', label='Validation IoU coefficient')
    plt.title('IoU Coefficient: '+model.name, fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.legend(prop={'size': 14})

    if fine:
        plt.savefig(DATA_ROOT+"plots/"+model.name+"_learning_curves.png")
    else:
        plt.savefig(DATA_ROOT+"plots/"+model.name+"_learning_curves_coarse.png")
    plt.show()

In [ ]:
plot_history(results, model)

### Evaluation

For evaluation IoU scores, we need to calculate the *hard* IoU score as opposed to the *soft* IoU score. We used soft IoU function for training loss. Also, we want to resize everything to the original resolution. 

See this [link](https://www.jeremyjordan.me/semantic-segmentation/)

In [ ]:
def evaluate_iou(model, dataset, n_samples):
    
    iou_scores = np.zeros((n_samples,))
    inf_times = np.zeros((n_samples, ))
    miou_op =  tf.keras.metrics.MeanIoU(num_classes=n_classes-1)
    
    for idx, (image, mask) in enumerate(dataset):
        print("\r Predicting {} \ {} ".format(idx+1, n_samples), end='')
        
        X = np.expand_dims(image.numpy(), axis=0)
        y_true = np.expand_dims(mask.numpy(), axis=0)
        
        t_start = time.time()
        y_pred = model.predict(X)
        t_end = time.time()
        t_inf = t_end-t_start
        
        inf_times[idx] = t_inf
        
        if "u2net" in model.name:
            y_pred = y_pred[0]
            
        y_pred = tf.image.resize(y_pred, (1024, 2048))
        threshold = tf.math.reduce_max(y_pred, axis=-1, keepdims=True)
        y_pred = tf.logical_and(y_pred >= threshold, tf.abs(y_pred) > 1e-12)
        
        y_pred = tf.cast(tf.squeeze(y_pred, axis=0), tf.int32)
        y_true = tf.cast(tf.squeeze(y_true, axis=0), tf.int32)
        
        y_true = tf.argmax(y_true[:,:,1:], axis=-1)
        y_pred = tf.argmax(y_pred[:,:,1:], axis=-1)
                
        miou_op.update_state(y_true, y_pred)
        iou = miou_op.result().numpy()
        iou_scores[idx] = iou
        
        if idx == (n_samples-1):
            break
    
    print("Average inference time: {:.2f}s".format(np.mean(inf_times)))
            
    return iou_scores, miou_op

In [ ]:
iou_scores, miou_op = evaluate_iou(model=model, dataset=eval, n_samples=TEST_LENGTH)
iou_mean = np.mean(iou_scores)

In [ ]:
union_int = np.sum(miou_op.get_weights()[0], axis=0)+np.sum(miou_op.get_weights()[0], axis=1)
inters = np.diag(miou_op.get_weights()[0])
ious = inters / (union_int-inters+1)

In [ ]:
plot_iou_trainId(
    trainId_label_map=trainid2label,
    catId_label_map=catId2label, 
    n_classes=n_classes, 
    iou_class=ious,
    model=model, 
    iou_mean=iou_mean,
    current_dir=DATA_ROOT
)

In [ ]:
def visualize_confusion_matrix(confusion, metric, label_classes, model):
    plt.figure(figsize=(13,13))
    plt.title("{} Confusion Matrix, with Mean IoU = {:.3f}".format(model.name, metric), fontsize=22)
    plt.imshow(confusion)
    ax, fig = plt.gca(), plt.gcf()
    plt.xticks(np.arange(len(label_classes)), label_classes)
    plt.yticks(np.arange(len(label_classes)), label_classes)
    # set horizontal alignment mode (left, right or center) and rotation mode(anchor or default)
    plt.setp(ax.get_xticklabels(), rotation=-90, ha="center", rotation_mode="default")
    # avoid top and bottom part of heatmap been cut
    ax.set_xticks(np.arange(len(label_classes) + 1) - .5, minor=True)
    ax.set_yticks(np.arange(len(label_classes) + 1) - .5, minor=True)
    ax.tick_params(which="minor", bottom=False, left=False)
    ax.grid(False)
    fig.tight_layout()
    plt.show()

In [ ]:
visualize_confusion_matrix(
    confusion = miou_op.get_weights()[0] / np.sum(miou_op.get_weights()[0], axis=0), 
    metric = iou_mean, 
    label_classes = [trainid2label[i].name for i in range(1, n_classes)],
    model = model
)